# Contents:
I. Outlier Treatment:

II. Feature Engineering:

Average price by sub-area:

Amenities score:

Average price by Amenities score:

III. Feature Extraction:

Wordcloud of the Description:

POS features:

Ngram features:

IV. Saving the final model ready data:

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

import matplotlib.pyplot as plt
import seaborn as sns

import os
import time 
import re

import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('data/news_share_data_selected.csv')
print(df.shape)
df.head()

(1000, 32)


,cleaned_title,cleaned_text,weekday,is_weekend,month,month number,weekday number,title_tokens,text_tokens,cleaned_data_channel,Channel_Entertainment,Channel_Lifestyle,Channel_Others,Channel_Social Media,Channel_Tech,unique_tokens_rate,num_hrefs,num_imgs,num_videos,average_token_length,num_keywords,min_avg_key,max_avg_key,avg_avg_key,href_avg_shares,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,title_subjectivity,title_sentiment_polarity,shares
0,time youtube competitor,reported last week yahoo getting online video ...,Monday,0,March,1,1,7,195,Business,0,0,0,0,0,0.640816,5,3,1,4.579592,5,1459.500,4817.692,3370.415,1833.333,0.403762,0.038762,0.036735,0.024490,0.25,0.25,1600
1,ultimate roundup 2014s biggest april fools day...,april fools marks day pranking goes corporate ...,Tuesday,0,April,0,5,12,108,Tech,0,0,0,0,1,0.549618,4,1,0,4.636704,7,0.000,3581.094,2556.367,8500.000,0.410707,0.180934,0.044944,0.003745,0.00,0.00,1500
2,selfie one garish jacket red sox white house,2013 world series champion boston red sox visi...,Tuesday,0,April,0,5,15,517,Tech,0,0,0,0,1,0.577236,8,1,0,4.627451,7,2270.889,3852.800,3018.167,2076.333,0.397545,0.044864,0.035294,0.019608,0.00,0.00,1400
3,african telecom proposes diverse emoticons,less week apple pledged make emoticons raciall...,Tuesday,0,April,0,5,6,283,Tech,0,0,0,0,1,0.525896,4,1,0,5.083665,9,0.000,3581.094,1871.403,2200.000,0.491578,0.157620,0.047809,0.011952,0.00,0.00,1300
4,google launch improved android camera app repo...,android users may soon see series new features...,Tuesday,0,April,0,5,10,282,Others,0,0,1,0,0,0.588889,6,9,0,4.336022,7,3581.094,8453.451,5155.323,805.000,0.628535,0.078497,0.053763,0.034946,0.00,0.00,3000


In [2]:
num_df = df.drop(columns = ['cleaned_title', 'cleaned_text','cleaned_data_channel','weekday','month'])
num_df.head(1)

,is_weekend,month number,weekday number,title_tokens,text_tokens,Channel_Entertainment,Channel_Lifestyle,Channel_Others,Channel_Social Media,Channel_Tech,unique_tokens_rate,num_hrefs,num_imgs,num_videos,average_token_length,num_keywords,min_avg_key,max_avg_key,avg_avg_key,href_avg_shares,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,title_subjectivity,title_sentiment_polarity,shares
0,0,1,1,7,195,0,0,0,0,0,0.640816,5,3,1,4.579592,5,1459.5,4817.692,3370.415,1833.333,0.403762,0.038762,0.036735,0.02449,0.25,0.25,1600


In [4]:
# Treating outliers in the numeric columns

print("\nDistribution (raw):")
display(num_df.describe())        

# Outlier treatment
def clip_outliers(df,col):
    q_l = df[col].min()
    q_h = df[col].quantile(0.95)
    df[col] = df[col].clip(lower = q_l, upper = q_h)
    return df

for col in num_df:
    df = clip_outliers(df,col)
    

print("\nDistribution (after outlier treatment):")
display(num_df.describe())  



Distribution (raw):


,is_weekend,month number,weekday number,title_tokens,text_tokens,Channel_Entertainment,Channel_Lifestyle,Channel_Others,Channel_Social Media,Channel_Tech,unique_tokens_rate,num_hrefs,num_imgs,num_videos,average_token_length,num_keywords,min_avg_key,max_avg_key,avg_avg_key,href_avg_shares,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,title_subjectivity,title_sentiment_polarity,shares
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.159000,0.001000,3.690000,10.097000,516.784000,0.185000,0.050000,0.350000,0.055000,0.196000,0.526981,11.737000,5.162000,1.218000,4.604362,7.207000,1133.532286,6425.997502,3355.163801,13541.256937,0.446699,0.106147,0.038162,0.017782,0.293414,0.057708,4243.046000
std,0.365859,0.031623,2.053806,2.392345,504.081848,0.388492,0.218054,0.477208,0.228094,0.397167,0.118838,15.200358,8.450631,4.085482,0.702689,1.858926,1181.809088,7341.729617,1456.128536,51574.105851,0.109301,0.092589,0.015680,0.010570,0.325319,0.266991,22641.282639
min,0.000000,0.000000,0.000000,5.000000,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,3568.870000,1360.101000,0.000000,0.000000,-0.300000,0.000000,0.000000,0.000000,-1.000000,58.000000
25%,0.000000,0.000000,2.000000,8.000000,211.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.466767,4.000000,1.000000,0.000000,4.490866,6.000000,0.000000,3613.040000,2534.983000,1300.000000,0.395563,0.051936,0.028663,0.011230,0.000000,0.000000,1000.000000
50%,0.000000,0.000000,4.000000,10.000000,355.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.533133,8.000000,2.000000,0.000000,4.685610,7.000000,984.533350,4636.361500,3006.859500,2766.667000,0.456005,0.102545,0.037354,0.016303,0.200000,0.000000,1600.000000
75%,0.000000,0.000000,5.000000,12.000000,642.750000,0.000000,0.000000,1.000000,0.000000,0.000000,0.596603,14.000000,7.000000,1.000000,4.856051,9.000000,2119.037500,6556.605000,3808.598000,7320.500000,0.509489,0.163003,0.046915,0.022603,0.500000,0.150000,3100.000000
max,1.000000,1.000000,6.000000,19.000000,4696.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.867925,162.000000,84.000000,51.000000,7.218430,10.000000,3613.040000,102200.000000,17604.270000,663600.000000,0.875000,0.489623,0.120805,0.103030,1.000000,1.000000,663600.000000



Distribution (after outlier treatment):


,is_weekend,month number,weekday number,title_tokens,text_tokens,Channel_Entertainment,Channel_Lifestyle,Channel_Others,Channel_Social Media,Channel_Tech,unique_tokens_rate,num_hrefs,num_imgs,num_videos,average_token_length,num_keywords,min_avg_key,max_avg_key,avg_avg_key,href_avg_shares,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,title_subjectivity,title_sentiment_polarity,shares
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.159000,0.001000,3.690000,10.097000,516.784000,0.185000,0.050000,0.350000,0.055000,0.196000,0.526981,11.737000,5.162000,1.218000,4.604362,7.207000,1133.532286,6425.997502,3355.163801,13541.256937,0.446699,0.106147,0.038162,0.017782,0.293414,0.057708,4243.046000
std,0.365859,0.031623,2.053806,2.392345,504.081848,0.388492,0.218054,0.477208,0.228094,0.397167,0.118838,15.200358,8.450631,4.085482,0.702689,1.858926,1181.809088,7341.729617,1456.128536,51574.105851,0.109301,0.092589,0.015680,0.010570,0.325319,0.266991,22641.282639
min,0.000000,0.000000,0.000000,5.000000,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,3568.870000,1360.101000,0.000000,0.000000,-0.300000,0.000000,0.000000,0.000000,-1.000000,58.000000
25%,0.000000,0.000000,2.000000,8.000000,211.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.466767,4.000000,1.000000,0.000000,4.490866,6.000000,0.000000,3613.040000,2534.983000,1300.000000,0.395563,0.051936,0.028663,0.011230,0.000000,0.000000,1000.000000
50%,0.000000,0.000000,4.000000,10.000000,355.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.533133,8.000000,2.000000,0.000000,4.685610,7.000000,984.533350,4636.361500,3006.859500,2766.667000,0.456005,0.102545,0.037354,0.016303,0.200000,0.000000,1600.000000
75%,0.000000,0.000000,5.000000,12.000000,642.750000,0.000000,0.000000,1.000000,0.000000,0.000000,0.596603,14.000000,7.000000,1.000000,4.856051,9.000000,2119.037500,6556.605000,3808.598000,7320.500000,0.509489,0.163003,0.046915,0.022603,0.500000,0.150000,3100.000000
max,1.000000,1.000000,6.000000,19.000000,4696.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.867925,162.000000,84.000000,51.000000,7.218430,10.000000,3613.040000,102200.000000,17604.270000,663600.000000,0.875000,0.489623,0.120805,0.103030,1.000000,1.000000,663600.000000
